<a href="https://colab.research.google.com/github/Siddharth02003/CS6910_assignment_3/blob/main/CS6910_assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.optim import Adam
import random
from tqdm import tqdm
import sys
from argparse import ArgumentParser
import numpy as np

In [2]:
!pip install wandb -qqq
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [3]:
!wandb login --relogin
entity_name="siddharth-s"

project_name="FODL_Assignment_3"

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
data=pd.read_csv("/content/drive/MyDrive/aksharantar_sampled/hin/hin_train.csv")
data

,shastragaar,शस्त्रागार
0,bindhya,बिन्द्या
1,kirankant,किरणकांत
2,yagyopaveet,यज्ञोपवीत
3,ratania,रटानिया
4,vaganyache,वागण्याचे
...,...,...
51194,toned,टोंड
51195,mutanaazaa,मुतनाज़ा
51196,asahmaton,असहमतों
51197,sulgaayin,सुलगायीं


In [4]:
'''
Dataset class is responsible for cleaning and processing the word data, 
and making a dictionary
'''
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, my_path, batch_size=1, my_lang="hin", my_type="train", start_of_word="\t", end_of_word="\n"):
        self.my_path = os.path.join(my_path, my_lang)
        self.my_path = list(map(lambda x: os.path.join(self.my_path, x), sorted(os.listdir(self.my_path))))
        self.start_of_word = start_of_word
        self.end_of_word = end_of_word
        self.batch_size = batch_size
        self.my_type = my_type.lower()

        self.data_x, self.data_y = None, None

        self.data = self.read_my_data()
        self.tokenize()
        self.process_data()

    def read_my_data(my_path: str, start_of_word, end_of_word):
        with open(my_path, "r") as f:
            lines = [line.split(",") for line in f.read().split("\n") if line != '']
            
        lines = [[start_of_word + s + end_of_word for s in line] for line in lines]
        return lines

    def read_my_data(self):
        self.test, self.train, self.val = [self.read_my_data(i, self.start_of_word, self.end_of_word) for i in self.my_path]

    def tokenize(self):
        x_tokens, y_tokens = set(), set()
        self.max_x_length, self.max_y_length = -1, -1

        for x, y in self.train:
            for ch in x:
                x_tokens.add(ch)
         
            for ch in y:
                y_tokens.add(ch)

        self.x_tokens, self.y_tokens = sorted(list(x_tokens)), sorted(list(y_tokens))

        self.x_length, self.y_length = len(self.x_tokens) + 2, len(self.y_tokens) + 2

        self.x_to_index = {ch: i + 2 for i, ch in enumerate(self.x_tokens)}
        self.y_to_index = {ch: i + 2 for i, ch in enumerate(self.y_tokens)}
        
        self.x_to_index[" "] = 0
        self.y_to_index[" "] = 0

        self.x_to_index["unk"] = 1
        self.y_to_index["unk"] = 1

        self.index_to_x = {i: char for char, i in self.x_to_index.items()}
        self.index_to_y = {i: char for char, i in self.y_to_index.items()}

    def process_data(self):
        for (x, y) in self.data:
            self.max_x_length = max(self.max_x_length, len(x))
            self.max_y_length = max(self.max_y_length, len(y)) 

        a = torch.zeros((len(self.data), self.max_x_length), dtype=torch.long)
        b = torch.zeros((len(self.data), self.max_y_length), dtype=torch.long)

        for i, [x, y] in enumerate(self.data):
            for j, ch in enumerate(x):
                if ch in self.x_to_index.keys():
                    a[i, j] = self.x_to_index[ch]
                else:
                    a[i, j] = self.x_to_index["unk"]

            a[i,j+1:] = self.x_to_index[" "]
            
            for j, ch in enumerate(y):
                if ch in self.y_to_index.keys():
                    b[i, j] = self.y_to_index[ch]
                else:
                    b[i, j] = self.y_to_index["unk"]

            b[i,j+1:] = self.y_to_index[" "]
        return a,b

In [ ]:
path = "/content/drive/MyDrive/aksharantar_sampled/"
data= dataset(path, typ="test")
dataloader = DataLoader(data, batch_size=32)
for trainx, trainy in dataloader:
      print(trainx.shape)
      print(trainy.shape)

In [ ]:
def calculate_word_accuracy(predictions: torch.Tensor, targets: torch.Tensor) -> float:
    predicted_indices = torch.argmax(predictions, dim=1)
    batch_size = targets.shape[0]

    with torch.no_grad():
        correct_count = 0
        for i in range(batch_size):
            if (predicted_indices[i] == targets[i]).sum().item() == targets.shape[1]:
                correct_count += 1
    
    return correct_count / batch_size

def calculate_char_accuracy(predictions: torch.Tensor, targets: torch.Tensor) -> float:
    num_chars = predictions.shape[1]
    batch_size = predictions.shape[0]

    with torch.no_grad():
        predicted_indices = torch.argmax(predictions, dim=1)
        correct_count = (predicted_indices == targets).sum().item()
  
    return correct_count / (num_chars * batch_size)


In [8]:
'''
The encoder and decoder classes take in arguments like 
input size
embedding size
hiddensize
numlayers
dropout
bidirectional
type of cell

all of these are hyperparameters as well tuned using wandb sweeps
'''
import torch
import torch.nn as nn
import sys
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(
        self,
        inputSize: int,
        embedSize: int,
        hiddenSize: int,
        numLayers: int,
        dropout: float,
        bidirectional: bool,
        typ: str
    ):
        super().__init__()
        self.hiddenSize = hiddenSize
        self.inputSize = inputSize
        self.numLayers = numLayers
        self.typ = typ.upper()
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(inputSize, embedSize)
        self.seq = self._get_sequence(typ, embedSize, hiddenSize, dropout, numLayers, bidirectional)

    def forward(self, input, hidden):
        embed = self.embedding(input)
        output = embed
        output, hidden = self.seq(output, hidden)
        return output, hidden

    def initHidden(self, batch_size: int, device: torch.device):
        if self.typ == "LSTM":
            return (
                torch.randn(
                    (1 + int(self.bidirectional)) * self.numLayers,
                    batch_size,
                    self.hiddenSize,
                    device=device,
                ),
                torch.randn(
                    (1 + int(self.bidirectional)) * self.numLayers,
                    batch_size,
                    self.hiddenSize,
                    device=device,
                ),
            )
        return torch.randn(
            (1 + int(self.bidirectional)) * self.numLayers,
            batch_size,
            self.hiddenSize,
            device=device,
        )

    def _get_sequence(
        self,
        typ: str,
        embedSize: int,
        hiddenSize: int,
        dropout: float,
        numLayers: int,
        bidirectional: bool,
    ):
        typ = typ.upper()
        if typ == "GRU":
            return nn.GRU(
                embedSize,
                hiddenSize,
                dropout=dropout,
                num_layers=numLayers,
                bidirectional=bidirectional,
                batch_first=True,
            )
        elif typ == "LSTM":
            return nn.LSTM(
                embedSize,
                hiddenSize,
                dropout=dropout,
                num_layers=numLayers,
                bidirectional=bidirectional,
                batch_first=True,
            )
        elif typ == "RNN":
            return nn.RNN(
                embedSize,
                hiddenSize,
                dropout=dropout,
                num_layers=numLayers,
                bidirectional=bidirectional,
                batch_first=True,
            )
        else:
            raise NotImplementedError


class Decoder(nn.Module):
    def __init__(
        self,
        outputSize: int,
        embedSize: int,
        hiddenSize: int,
        numLayers: int,
        dropout: float,
        bidirectional: bool,
        typ: str,
    ):
        super().__init__()
        self.hiddenSize = hiddenSize
        self.outputSize = outputSize
        self.numLayers = numLayers
        self.typ = typ.upper()
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(outputSize, embedSize)
        self.seq = self._get_sequence(typ, embedSize, hiddenSize, dropout, numLayers, bidirectional)
        self.out = nn.Linear((1 + int(self.bidirectional)) * self.hiddenSize, self.outputSize)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input: torch.Tensor, hidden: torch.Tensor):
        inp = input.unsqueeze(1)
        embed = self.embedding(inp)
        output = embed
        output, hidden = self.seq(output, hidden)
        output = self.softmax(F.relu(self.out(output)))
        return output, hidden

    def initHidden(self, batch_size: int, device: torch.device):
        if self.typ == "LSTM":
            return (
                torch.randn(
                    (1 + int(self.bidirectional)) * self.numLayers,
                    batch_size,
                    self.hiddenSize,
                    device=device,
                ),
                torch.randn(
                    (1 + int(self.bidirectional)) * self.numLayers,
                    batch_size,
                    self.hiddenSize,
                    device=device,
                ),
            )
        return torch.randn(
            (1 + int(self.bidirectional)) * self.numLayers,
            batch_size,
            self.hiddenSize,
            device=device,
        )

    def _get_sequence(
        self,
        typ: str,
        embedSize: int,
        hiddenSize: int,
        dropout: float,
        numLayers: int,
        bidirectional: bool,
    ):
        typ = typ.upper()
        if typ == "GRU":
            return nn.GRU(
                embedSize,
                hiddenSize,
                dropout=dropout,
                num_layers=numLayers,
                bidirectional=bidirectional,
                batch_first=True,
            )
        elif typ == "LSTM":
            return nn.LSTM(
                embedSize,
                hiddenSize,
                dropout=dropout,
                num_layers=numLayers,
                bidirectional=bidirectional,
                batch_first=True,
            )
        elif typ == "RNN":
            return nn.RNN(
                embedSize,
                hiddenSize,
                dropout=dropout,
                num_layers=numLayers,
                bidirectional=bidirectional,
                batch_first=True,
            )
        else:
            raise Wrongcelltype

'class Encoder(nn.Module):\n    def __init__(\n        self,\n        inputSize: int,\n        embedSize: int,\n        hiddenSize: int,\n        numLayers: int,\n        dropout: float,\n        bidirectional: bool,\n        typ: str\n    ):\n        super().__init__()\n        self.hiddenSize = hiddenSize\n        self.inputSize = inputSize\n        self.numLayers = numLayers\n        self.typ = typ.upper()\n        self.bidirectional = bidirectional\n\n        self.embedding = nn.Embedding(inputSize, embedSize)\n        self.seq = self._get_sequence(typ, embedSize, hiddenSize, dropout, numLayers, bidirectional)\n\n    def forward(self, input, hidden):\n        embed = self.embedding(input)\n        output = embed\n        output, hidden = self.seq(output, hidden)\n        return output, hidden\n\n    def initHidden(self, batch_size: int, device: torch.device):\n        if self.typ == "LSTM":\n            return (\n                torch.randn(\n                    (1 + int(self.bidir

In [ ]:
teacherForcingRatio=0.0
language='hin'
batch_size=32
teacherForcingRatio = teacherForcingRatio


trainData = dataset(path, lang=language, typ="train")
valData = dataset(path, lang=language, typ="val")
testData = dataset(path, lang=language, typ="test")

trainLoader = DataLoader(trainData, batch_size=batch_size)
valLoader = DataLoader(valData, batch_size=batch_size)
testLoader = DataLoader(testData, batch_size=batch_size)

sowToken = trainData.x2TDict["\t"]
eowToken = trainData.x2TDict["\n"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Training is happening on: {device}")

In [10]:
 
'''
The loss_calculation function integrates the teacher forcing functionality, if teacher forcing isnt applicable it starts feeding its previous output as the new input
for the model. 
returns the loss and word, char accuracies respectively
'''
def calc_loss(input_tensor: torch.Tensor, target_tensor: torch.Tensor,
                   encoder: Encoder, decoder: Decoder,
                   enc_optimizer: torch.optim, dec_optimizer: torch.optim,
                   criterion: nn.Module, max_len: int, use_attention: bool):

    enc_hidden = encoder.init_hidden(batch_size, device)

    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()

    input_len = input_tensor.size(1)
    target_len = target_tensor.size(1)

    enc_outputs = torch.zeros(max_len, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_len):
        enc_output, enc_hidden = encoder(input_tensor, enc_hidden)

    dec_hidden = enc_hidden
    dec_input = target_tensor[:, 0]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    outputs = torch.zeros(batch_size, target_len, train_data.y_len).to(device)

    if use_teacher_forcing:
        if use_attention:
            for di in range(1, target_len):
                dec_output, dec_hidden, decoder_attention = decoder(dec_input, dec_hidden, enc_outputs)

                dec_input = target_tensor[:, di].unsqueeze(0)
        else:
            for di in range(1, target_len):
                dec_output, dec_hidden = decoder(dec_input, dec_hidden)

                outputs[:, di] = dec_output.squeeze(1)

                dec_input = target_tensor[:, di]
    else:
        if use_attention:
            for di in range(target_len):
                dec_output, dec_hidden = decoder(dec_input, dec_hidden, enc_outputs)

                outputs[:, di] = dec_output.squeeze(1)

                top = dec_output.argmax(-1)
                dec_input = top.squeeze(1).detach()

                if dec_input.item() == eow_token:
                    break
        else:
            for di in range(target_len):
                dec_output, dec_hidden = decoder(dec_input, dec_hidden)

                outputs[:, di] = dec_output.squeeze(1)

                top = dec_output.argmax(-1)
                dec_input = top.squeeze(1).detach()

    outputs = outputs.permute(0, 2, 1)

    loss += criterion(outputs, target_tensor)
    loss.backward()

    enc_optimizer.step()
    dec_optimizer.step()

    accuracy_word = calculate_word_accuracy(outputs, target_tensor)
    accuracy_char = calculate_char_accuracy(outputs, target_tensor)

    return loss.item(), accuracy_word, accuracy_char


Training is happening on: cuda


In [ ]:
'''
The function evaluate takes the encoder decoder and target prediction pair is arguments
it returns the char and word accuracies for the corresponding pairs of target, preds
'''

def evaluate(encoder: Encoder, decoder: Decoder, pair: list, criterion: nn.Module):
    with torch.no_grad():
        input_tensor = pair[0].to(device)
        target_tensor = pair[1].to(device)

        input_len = input_tensor.size(1)
        target_len = target_tensor.size(1)

        enc_hidden = encoder.init_hidden(batch_size, device)

        outputs = torch.zeros(batch_size, target_len, train_data.y_len).to(device)

        loss = 0
        for ei in range(input_len):
            enc_output, enc_hidden = encoder(input_tensor, enc_hidden)

        dec_hidden = enc_hidden
        dec_input = target_tensor[:, 0]

        for di in range(target_len):
            dec_output, dec_hidden = decoder(dec_input, dec_hidden)

            outputs[:, di] = dec_output.squeeze(1)

            top = dec_output.argmax(-1)
            dec_input = top.squeeze(1).detach()

        outputs = outputs.permute(0, 2, 1)

        loss += criterion(outputs, target_tensor)

        accuracy_word = calculate_word_accuracy(outputs, target_tensor)
        accuracy_char = calculate_char_accuracy(outputs, target_tensor)

    return loss.item(), accuracy_word, accuracy_char


In [12]:

sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'Validation_char_Accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'inp_embed_size': {
            'values': [32,64,256]
        },
        'num_enc_layers' : {
           'values' : [1,2,3,4]
        },
        'num_dec_layers' : {
           'values' : [1,2,3,4]
        },
        'hidden_size': {
            'values': [256,512,1024]
        },
        'cell_type': {
            'values': ['RNN','LSTM','GRU'],
        },
        'bidirectional': {
            'values': [True,False]
        },
        'dropout': {
            'values': [0.25,0.3,0.4]
        }
    }
}

In [15]:
def train(dropout=0.25,bidirectional=False,cell_type='GRU',hidden_size=512,num_dec_layers=1,num_enc_layers=1,inp_embed_size=256,useAttn=False ,print_every=1, learning_rate=0.001, visualize=False):
    
    config_defaults={
      'inp_embed_size': 256,
      'num_enc_layers' : 1,
      'num_dec_layers' : 1,
      'hidden_size': 512,
      'cell_type': 'GRU',
      'bidirectional': False,
      'dropout': 0.25
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config

    hiddenSize = config.hidden_size
    epochs = 7
    backbone = config.cell_type
    embedSize = config.inp_embed_size
    num_enc_layers= config.num_enc_layers
    num_dec_layers= config.num_dec_layers
    dropout= config.dropout
    bidirectional= config.bidirectional

    path = "/content/drive/MyDrive/aksharantar_sampled/"
    teacherForcingRatio=0.0
    language='hin'
    batch_size=32 
    teacherForcingRatio = 0.0

    trainData = dataset(path, lang=language, typ="train")
    valData = dataset(path, lang=language, typ="val")
    testData = dataset(path, lang=language, typ="test")

    trainLoader = DataLoader(trainData, batch_size=batch_size)
    valLoader = DataLoader(valData, batch_size=batch_size)
    testLoader = DataLoader(testData, batch_size=batch_size)

    sowToken = trainData.x2TDict["\t"] #hardcoded
    eowToken = trainData.x2TDict["\n"] #hardcoded

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    inputSize = trainData.xLen
    outputSize = trainData.yLen

    encoder = Encoder(inputSize, embedSize, hiddenSize, num_enc_layers, dropout, bidirectional, backbone).to(device)
    decoder = Decoder(outputSize, embedSize, hiddenSize, num_enc_layers, dropout, bidirectional, backbone).to(device)
    print_loss_total = 0  # Reset every print_every
    

    encOptim = Adam(encoder.parameters(), lr=learning_rate)
    decOptim = Adam(decoder.parameters(), lr=learning_rate) 

    criterion = nn.NLLLoss()

    for epoch in range(1, epochs + 1):
        loop_obj = tqdm(trainLoader)
        epochTrainAccW, epochTrainAccC,epochTrainLoss = [], [], []
        for training_pair in loop_obj:
            inputTensor = training_pair[0].to(device)
            targetTensor = training_pair[1].to(device)

            loss, accW, accC = calc_loss(inputTensor, targetTensor, encoder, decoder, encOptim, decOptim, criterion, inputSize, useAttn)
            

            loop_obj.set_postfix({f"Loss": f"{loss:0.3f}"})
            loop_obj.set_description(f"Epoch {epoch}")
            
            epochTrainLoss.append(loss)
            epochTrainAccW.append(accW)
            epochTrainAccC.append(accC)

        print(f"Training Loss is {sum(epochTrainLoss)/len(epochTrainLoss)}")        
        print(f"Training Word Accuracy is {sum(epochTrainAccW)/len(epochTrainAccW)}")
        print(f"Training Character Accuracy is {sum(epochTrainAccC)/len(epochTrainAccC)}")

        epochValLoss, epochValAccW, epochValAccC = [],  [], []

        if epoch % print_every == 0:
            with torch.no_grad():
                print("Starting Validation")

                for pair in tqdm(valLoader):
                    loss, accW, accC = evaluate(encoder, decoder, pair, criterion)

                    epochValLoss.append(loss)
                    epochValAccW.append(accW)
                    epochValAccC.append(accC)
                name_run="run_attn" + '_' + str(config.cell_type) + '_' +str(config.hidden_size) + '_' + str(config.inp_embed_size)
                wandb.run.name = name_run
                wandb_log=True
                log_dict = {"Train_loss": np.min(epochTrainLoss), "Validation_loss": np.min(epochValLoss), "Train_char_Accuracy": np.max(epochTrainAccC), "Validation_char_Accuracy":np.max(epochValAccC)}
                wandb.log(log_dict)
                print(f"Validation Loss is {sum(epochValLoss)/len(epochValLoss)}")        
                print(f"Validation Word Accuracy is {sum(epochValAccW)/len(epochValAccW)}")
                print(f"Validation Character Accuracy is {sum(epochValAccC)/len(epochValAccC)}")
    wandb.run.save()
    wandb.run.finish()
    return encoder, decoder

In [ ]:
def do_sweep(project_name,entity_name,dropout=0.25,bidirectional=False,cell_type='GRU',hidden_size=512,num_dec_layers=1,num_enc_layers=1,inp_embed_size=256,useAttn=False ):
  sweep_id=wandb.sweep(sweep_config, entity=entity_name, project=project_name)
  wandb.agent(sweep_id, train)

In [16]:
encoder, decoder= train()

wandb: Currently logged in as: na20b064 (siddharth-s). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Epoch 1: 100%|██████████| 1600/1600 [03:41<00:00,  7.23it/s, Loss=0.981]


Training Loss is 1.309193760752678
Training Word Accuracy is 0.0
Training Character Accuracy is 0.21380256204044099
Starting Validation


100%|██████████| 128/128 [00:04<00:00, 27.59it/s]


Validation Loss is 0.8467504139989614
Validation Word Accuracy is 0.0
Validation Character Accuracy is 0.24663947610294112


...

Epoch 7: 100%|██████████| 1600/1600 [03:32<00:00,  7.55it/s, Loss=0.427]


Training Loss is 0.5153965140134096
Training Word Accuracy is 0.12873046875
Training Character Accuracy is 0.2722814223345588
Starting Validation


100%|██████████| 128/128 [00:04<00:00, 27.58it/s]
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Validation Loss is 0.46606109640561044
Validation Word Accuracy is 0.158935546875
Validation Character Accuracy is 0.2777063706341911


Train_char_Accuracy,▁▃▄▅▅▆█
Train_loss,█▆▅▅▄▄▁
Validation_char_Accuracy,▁▃▃▄▄▄█
Validation_loss,█▆▅▅▅▅▁
Train_char_Accuracy,0.29366
Train_loss,0.29305
Validation_char_Accuracy,0.29458
Validation_loss,0.27808


In [18]:
def evaluate_test(encoder: Encoder, decoder: Decoder, criterion: nn.Module):
   path = "/content/drive/MyDrive/aksharantar_sampled/"
   language='hin'
   testData = dataset(path, lang=language, typ="test")
   testLoader = DataLoader(testData, batch_size=32)
   outputlist=[]
   AccWlist=[] 
   AccClist=[]

   for pair in tqdm(testLoader):  
     with torch.no_grad():
        inputTensor = pair[0].to(device)
        targetTensor = pair[1].to(device)

        inputLen = inputTensor.size(1)
        targetLen = targetTensor.size(1)
        
        encHidden = encoder.initHidden(batch_size, device)
        
        outputs = torch.zeros(batch_size, targetLen, trainData.yLen).to(device)

        loss = 0
        for ei in range(inputLen):
            encOutput, encHidden = encoder(inputTensor, encHidden)
            #encOutputs[ei] += encOutput[0, 0]

        decHidden = encHidden
        decInput = targetTensor[:, 0]

        for di in range(targetLen):
            decOutput, decHidden = decoder(decInput, decHidden)

            outputs[:, di] = decOutput.squeeze(1)

            top= decOutput.argmax(-1)
            decInput = top.squeeze(1).detach() 

        outputs = outputs.permute(0, 2, 1)
        
        loss += criterion(outputs, targetTensor)
        
        AccW = wordAccuracy(outputs, targetTensor)
        AccC = charAccuracy(outputs, targetTensor)
     AccWlist.append(AccW) 
     AccClist.append(AccC)
     outputlist.append(outputs)
   return outputlist, AccWlist, AccClist

In [20]:
outputlist, AccWlist, AccClist=evaluate_test(encoder,decoder,nn.NLLLoss())

100%|██████████| 128/128 [00:06<00:00, 19.15it/s]


In [24]:
np.max(AccWlist)

0.3125

Attention Module

In [ ]:
'''
The encoder decoder classes with attention applied
the encoder architecture remains same with decoder architecture having attention applied
across encoder outputs corresponding to each layer
'''


class Encoder(nn.Module):
    def __init__(self, inputSize, embedSize, hiddenSize, numLayers, dropout, bidirectional, typ):
        super().__init__()
        self.hiddenSize = hiddenSize
        self.inputSize = inputSize
        self.numLayers = numLayers
        self.typ = typ.upper()
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(inputSize, embedSize)

        if self.typ == "GRU":
            self.seq = nn.GRU(embedSize, hiddenSize, dropout=dropout, num_layers=numLayers,
                              bidirectional=bidirectional, batch_first=True)
        elif self.typ == "LSTM":
            self.seq = nn.LSTM(embedSize, hiddenSize, dropout=dropout, num_layers=numLayers,
                               bidirectional=bidirectional, batch_first=True)
        elif self.typ == "RNN":
            self.seq = nn.RNN(embedSize, hiddenSize, dropout=dropout, num_layers=numLayers,
                              bidirectional=bidirectional, batch_first=True)
        else:
            raise NotImplementedError

    def forward(self, input, hidden):
        embed = self.embedding(input)
        output = embed

        output, hidden = self.seq(output, hidden)

        return output, hidden

    def initHidden(self, batch_size, device):
        if self.typ == "LSTM":
            return (torch.randn((1 + int(self.bidirectional)) * self.numLayers, batch_size, self.hiddenSize,
                                device=device), torch.randn((1 + int(self.bidirectional)) * self.numLayers, batch_size,
                                                           self.hiddenSize, device=device))

        return torch.randn((1 + int(self.bidirectional)) * self.numLayers, batch_size, self.hiddenSize, device=device)


class AttentionDecoder(nn.Module):
    def __init__(self, outputSize, embedSize, hiddenSize, numLayers, dropout, bidirectional, typ):
        super().__init__()
        self.hiddenSize = hiddenSize
        self.outputSize = outputSize
        self.numLayers = numLayers
        self.typ = typ.upper()
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(outputSize, embedSize)
        self.attn = nn.Linear(hiddenSize + embedSize, hiddenSize)
        self.attn_combine = nn.Linear(hiddenSize + embedSize, hiddenSize)

        if self.typ == "GRU":
            self.seq = nn.GRU(hiddenSize, hiddenSize, dropout=dropout, num_layers=numLayers,
                              bidirectional=bidirectional, batch_first=True)
        elif self.typ == "LSTM":
            self.seq = nn.LSTM(hiddenSize, hiddenSize, dropout=dropout, num_layers=numLayers,
                               bidirectional=bidirectional, batch_first=True)
        elif self.typ == "RNN":
            self.seq = nn.RNN(hiddenSize, hiddenSize, dropout=dropout, num_layers=numLayers,
                              bidirectional=bidirectional, batch_first=True)
        else:
            raise NotImplementedError

        self.out = nn.Linear((1 + int(self.bidirectional)) * hiddenSize, outputSize)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input, hidden, encoder_outputs):
        embed = self.embedding(input)
        attn_weights = F.softmax(self.attn(torch.cat((embed, hidden[-1]), dim=1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)

        output = torch.cat((embed, attn_applied[:, 0]), dim=1)
        output = self.attn_combine(output).unsqueeze(1)

        output, hidden = self.seq(output, hidden)

        output = self.softmax(F.relu(self.out(output.squeeze(1))))

        return output, hidden

    def initHidden(self, batch_size, device):
        if self.typ == "LSTM":
            return (torch.randn((1 + int(self.bidirectional)) * self.numLayers, batch_size, self.hiddenSize,
                                device=device), torch.randn((1 + int(self.bidirectional)) * self.numLayers, batch_size,
                                                           self.hiddenSize, device=device))

        return torch.randn((1 + int(self.bidirectional)) * self.numLayers, batch_size, self.hiddenSize, device=device)


In [ ]:
'''
Same train function but with attention argument as True
'''
def train_withattn(useAttn=True ,print_every=1, learning_rate=0.001, visualize=False):
    
    config_defaults={
      'inp_embed_size': 256,
      'num_enc_layers' : 2,
      'num_dec_layers' : 2,
      'hidden_size': 512,
      'cell_type': 'GRU',
      'bidirectional': False,
      'dropout': 0.25
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config

    hiddenSize = config.hidden_size
    epochs = 5
    backbone = config.cell_type
    embedSize = config.inp_embed_size
    num_enc_layers= config.num_enc_layers
    num_dec_layers= config.num_dec_layers
    dropout= config.dropout
    bidirectional= config.bidirectional

    path = "/content/drive/MyDrive/aksharantar_sampled/"
    teacherForcingRatio=0.0
    language='hin'
    batch_size=32 
    teacherForcingRatio = 0.0

    trainData = dataset(path, lang=language, typ="train")
    valData = dataset(path, lang=language, typ="val")
    testData = dataset(path, lang=language, typ="test")

    trainLoader = DataLoader(trainData, batch_size=batch_size)
    valLoader = DataLoader(valData, batch_size=batch_size)
    testLoader = DataLoader(testData, batch_size=batch_size)

    sowToken = trainData.x2TDict["\t"] #hardcoded
    eowToken = trainData.x2TDict["\n"] #hardcoded

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    inputSize = trainData.xLen
    outputSize = trainData.yLen

    encoder = Encoder(inputSize, embedSize, hiddenSize, num_enc_layers, dropout, bidirectional, backbone).to(device)
    decoder = AttentionDecoder(outputSize, hiddenSize, num_enc_layers, inputSize, dropout, bidirectional, backbone).to(device)
    print_loss_total = 0  # Reset every print_every

    encOptim = Adam(encoder.parameters(), lr=learning_rate)
    decOptim = Adam(decoder.parameters(), lr=learning_rate) 

    criterion = nn.NLLLoss()

    for epoch in range(1, epochs + 1):
        loop_obj = tqdm(trainLoader)
        epochTrainAccW, epochTrainAccC,epochTrainLoss = [], [], []
        for training_pair in loop_obj:
            inputTensor = training_pair[0].to(device)
            targetTensor = training_pair[1].to(device)

            loss, accW, accC = calc_loss(inputTensor, targetTensor, encoder, decoder, encOptim, decOptim, criterion, inputSize, useAttn)
            

            loop_obj.set_postfix({f"Loss": f"{loss:0.3f}"})
            loop_obj.set_description(f"Epoch {epoch}")
            
            epochTrainLoss.append(loss)
            epochTrainAccW.append(accW)
            epochTrainAccC.append(accC)

        print(f"Training Loss is {sum(epochTrainLoss)/len(epochTrainLoss)}")        
        print(f"Training Word Accuracy is {sum(epochTrainAccW)/len(epochTrainAccW)}")
        print(f"Training Character Accuracy is {sum(epochTrainAccC)/len(epochTrainAccC)}")

        epochValLoss, epochValAccW, epochValAccC = [],  [], []

        if epoch % print_every == 0:
            with torch.no_grad():
                print("Starting Validation")

                for pair in tqdm(valLoader):
                    loss, accW, accC = evaluate(encoder, decoder, pair, criterion)

                    epochValLoss.append(loss)
                    epochValAccW.append(accW)
                    epochValAccC.append(accC)
                name_run="run_attn" + '_' + str(config.cell_type) + '_' +str(config.hidden_size) + '_' + str(config.inp_embed_size)
                wandb.run.name = name_run
                wandb_log=True
                log_dict = {"Train_loss": np.min(epochTrainLoss), "Validation_loss": np.min(epochValLoss), "Train_char_Accuracy": np.max(epochTrainAccC), "Validation_char_Accuracy":np.max(epochValAccC)}
                wandb.log(log_dict)
                print(f"Validation Loss is {sum(epochValLoss)/len(epochValLoss)}")        
                print(f"Validation Word Accuracy is {sum(epochValAccW)/len(epochValAccW)}")
                print(f"Validation Character Accuracy is {sum(epochValAccC)/len(epochValAccC)}")
    wandb.run.save()
    wandb.run.finish()

In [ ]:
train_withattn()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/1600 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity=entity_name, project=project_name)
wandb.agent(sweep_id, train_withattn)

Create sweep with ID: 2vyo58go
Sweep URL: https://wandb.ai/siddharth-s/FODL_Assignment_3/sweeps/2vyo58go


wandb: Agent Starting Run: obzjld3l with config:
wandb: 	bidirectional: False
wandb: 	cell_type: GRU
wandb: 	dropout: 0.25
wandb: 	hidden_size: 256
wandb: 	inp_embed_size: 64
wandb: 	num_dec_layers: 4
wandb: 	num_enc_layers: 1


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/1600 [00:00<?, ?it/s]


Run obzjld3l errored: RuntimeError('Sizes of tensors must match except in dimension 1. Expected size 1 but got size 32 for tensor number 1 in the list.')
wandb: ERROR Run obzjld3l errored: RuntimeError('Sizes of tensors must match except in dimension 1. Expected size 1 but got size 32 for tensor number 1 in the list.')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 82cmbzal with config:
wandb: 	bidirectional: True
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.25
wandb: 	hidden_size: 512
wandb: 	inp_embed_size: 256
wandb: 	num_dec_layers: 2
wandb: 	num_enc_layers: 1
wandb: Ctrl + C detected. Stopping sweep.
